In [1]:
import torch
from torch_geometric.datasets import OPFDataset
import pandas
import scipy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random
import ipywidgets as widgets
from IPython.display import display

In [2]:
num_samples = 10000
num_buses = 118
num_loads = 99
num_gens = 54

# PF delta

In [3]:
## Load pfdelta data
path_to_data = "/Users/albanpuech/benchmark_data/pfdelta/30000opfv2dataset_case118_ieee.mat"
pfdelta_data = scipy.io.loadmat(path_to_data)

In [4]:
pfdelta_data['bus_type'].shape

(118, 30000)

In [5]:
## Load pfdelta data
path_to_data = "/Users/albanpuech/benchmark_data/pfdelta/30000opfv2dataset_case118_ieee.mat"
pfdelta_data = scipy.io.loadmat(path_to_data)
baseMVA = pfdelta_data["baseMVA"]
print(baseMVA.dtype)
pd_ = pfdelta_data["bus"][:, 2]
qd_ = pfdelta_data["bus"][:, 3]
load_bus_mask = (pd_ > 0) | (qd_ > 0)
gen_bus = (
    torch.tensor(pfdelta_data["gen"][:, 0], dtype=torch.long) - 1
)  # change index of bus back to
gen_bus_expanded = gen_bus.expand(num_samples, -1)
spv_bus_mask = torch.tensor(pfdelta_data["bus"][:, 1] != 1, dtype=torch.bool)

# Get variables (only first 10000 samples)
pd = np.real(pfdelta_data["Dem"][load_bus_mask, :10000].T) / 100
qd = np.imag(pfdelta_data["Dem"][load_bus_mask, :10000].T) / 100
pg = torch.tensor(np.real(pfdelta_data["Gen"][:10000, :].T) / 100)
qg = torch.tensor(np.imag(pfdelta_data["Gen"][:10000, :].T) / 100)
vm = np.abs(pfdelta_data["Vol"][:10000, :].T)

pg_full = torch.zeros((num_samples, num_buses), dtype=pg.dtype)
qg_full = torch.zeros((num_samples, num_buses), dtype=qg.dtype)
pg_full.scatter_add_(1, gen_bus_expanded, pg)
qg_full.scatter_add_(1, gen_bus_expanded, qg)
pg = pg_full[:, spv_bus_mask]
qg = qg_full[:, spv_bus_mask]

perturbpf = {"pd": pd, "qd": qd, "pg": pg, "qg": qg, "vm": vm}

uint8


In [6]:
# Load OPFData
root = ".//Users/albanpuech/benchmark_data/opfdata"
dataset = OPFDataset(
    root=root,
    split="train",
    case_name="pglib_opf_case118_ieee",
    num_groups=1,
    topological_perturbations=True,
    force_reload=False,
)

pg_full = torch.zeros((num_samples, num_buses))
qg_full = torch.zeros((num_samples, num_buses))
pd = torch.zeros((num_samples, num_loads))
qd = torch.zeros((num_samples, num_loads))
vm = torch.zeros((num_samples, num_buses))

for i in range(10000):
    pg = dataset[i]["generator"]["y"][:, 0]
    qg = dataset[i]["generator"]["y"][:, 1]
    gen_bus = dataset[i]["generator", "generator_link", "bus"]["edge_index"][1, :]
    pg_full[i, :].scatter_add_(0, gen_bus, pg)
    qg_full[i, :].scatter_add_(0, gen_bus, qg)
    pd[i, :] = dataset[i]["load"]["x"][:, 0]
    qd[i, :] = dataset[i]["load"]["x"][:, 1]
    vm[i, :] = dataset[i]["bus"]["y"][:, 1]

pg = pg_full[:, spv_bus_mask]
qg = qg_full[:, spv_bus_mask]

opfdata = {"pd": pd, "qd": qd, "pg": pg, "qg": qg, "vm": vm}

In [7]:
# Load OPFLearn Data
opflearn_data_path = "/Users/albanpuech/benchmark_data/opflearn/pglib_opf_case118_ieee.csv"
df = pandas.read_csv(opflearn_data_path)
pd = df.filter(regex=r"^load\d+:pl$", axis=1).to_numpy()
qd = df.filter(regex=r"^load\d+:ql$", axis=1).to_numpy()
pg = torch.tensor(df.filter(regex=r"^gen\d+:pg$", axis=1).to_numpy())
qg = torch.tensor(df.filter(regex=r"^gen\d+:qg$", axis=1).to_numpy())
vm_df = df.filter(regex=r"^bus\d+:v_bus$", axis=1)


def safe_complex(val):
    if isinstance(val, str):
        val = val.replace(" + ", "+").replace(" - ", "-")  # fix spacing for conversion
        return complex(val)
    else:
        return np.nan


# Apply transformation
vm_complex = vm_df.applymap(safe_complex).to_numpy()
vm = np.abs(vm_complex)

# expand pg and qg
pg_full = torch.zeros((10000, 118), dtype=pg.dtype)
qg_full = torch.zeros((10000, 118), dtype=pg.dtype)
pg_full.scatter_add_(1, gen_bus_expanded, pg)
qg_full.scatter_add_(1, gen_bus_expanded, qg)
pg = pg_full[:, spv_bus_mask]
qg = qg_full[:, spv_bus_mask]

opflearn = {"pd": pd, "qd": qd, "pg": pg, "qg": qg, "vm": vm}

/var/folders/ld/19f7b26x5s9771w9ggsv_vxh0000gn/T/ipykernel_80455/1021886785.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  vm_complex = vm_df.applymap(safe_complex).to_numpy()


In [8]:
# Load datakit data
# No generator cost permutation - random topology perturbation
datakit_data_path = "/Users/albanpuech/datakit-data/case118/pf_random_topology_no_gencost/pf_node.csv"
df_dk_no_gc = pandas.read_csv(datakit_data_path)
df_dk_no_gc = df_dk_no_gc[(df_dk_no_gc["scenario"] < 10000)]  # only 10000 samples
for col in ["Pd", "Qd", "Pg", "Qg"]:
    df_dk_no_gc[col] = df_dk_no_gc[col] / 100
df_dk_no_gc = df_dk_no_gc.rename(
    columns={"Pd": "pd", "Qd": "qd", "Pg": "pg", "Qg": "qg", "Vm": "vm", "Va": "va"}
)  # rename to match other data

# Need to group by generator bus and load bus indices first then select bus indices to plot

In [11]:
# Generator cost permutation - no topology perturbation
datakit_gc_no_topology_path = "/Users/tamaragovindasamy/Desktop/gridfm-datakit/data_out/data_comparison/case118_ieee_gc_no_topology/raw/pf_node.csv"
datakit_gc_no_topology = pandas.read_csv(datakit_gc_no_topology_path)
datakit_gc_no_topology = datakit_gc_no_topology[
    (datakit_gc_no_topology["scenario"] < 10000)
]  # only 10000 samples
for col in ["Pd", "Qd", "Pg", "Qg"]:
    datakit_gc_no_topology[col] = datakit_gc_no_topology[col] / 100
datakit_gc_no_topology = datakit_gc_no_topology.rename(
    columns={"Pd": "pd", "Qd": "qd", "Pg": "pg", "Qg": "qg", "Vm": "vm", "Va": "va"}
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tamaragovindasamy/Desktop/gridfm-datakit/data_out/data_comparison/case118_ieee_gc_no_topology/raw/pf_node.csv'

In [9]:
# Generator cost permutation - random topology perturbation
datakit_gc_random_topology_path = "/Users/albanpuech/datakit-data/case118/pf_topology_random_gc_permute_agg_load/pf_node.csv"
datakit_gc_random_topology = pandas.read_csv(datakit_gc_random_topology_path)
datakit_gc_random_topology = datakit_gc_random_topology[
    (datakit_gc_random_topology["scenario"] < 10000)
]  # only 10000 samples
for col in ["Pd", "Qd", "Pg", "Qg"]:
    datakit_gc_random_topology[col] = datakit_gc_random_topology[col] / 100
datakit_gc_random_topology = datakit_gc_random_topology.rename(
    columns={"Pd": "pd", "Qd": "qd", "Pg": "pg", "Qg": "qg", "Vm": "vm", "Va": "va"}
)

In [10]:
# Contingency - Generator cost permutation - random topology perturbation
datakit_gc_random_topology_contingency_path = "/Users/albanpuech/datakit-data/case118/contingency_topology_random_gc_permuted/pf_node.csv"
datakit_gc_random_topology_contingency = pandas.read_csv(
    datakit_gc_random_topology_contingency_path
)
datakit_gc_random_topology_contingency = datakit_gc_random_topology_contingency[
    (datakit_gc_random_topology_contingency["scenario"] < 10000)
]  # only 10000 samples
for col in ["Pd", "Qd", "Pg", "Qg"]:
    datakit_gc_random_topology_contingency[col] = (
        datakit_gc_random_topology_contingency[col] / 100
    )
datakit_gc_random_topology_contingency = datakit_gc_random_topology_contingency.rename(
    columns={"Pd": "pd", "Qd": "qd", "Pg": "pg", "Qg": "qg", "Vm": "vm", "Va": "va"}
)

In [11]:
# Load PgLearn data
pglearn_data_path = (
    "/Users/albanpuech/benchmark_data/pglearn/pglearn.csv"
)
df_pg = pandas.read_csv(pglearn_data_path)

df_pg = df_pg[(df_pg["scenario"] < 10000)]  # only 10000 samples
for col in ["Pd", "Qd", "Pg", "Qg"]:
    df_pg[col] = df_pg[col] / 100
df_pg = df_pg.rename(
    columns={"Pd": "pd", "Qd": "qd", "Pg": "pg", "Qg": "qg", "Vm": "vm", "Va": "va"}
)  # rename to match other data

In [12]:
def plotting_function(
    feature, sample_buses, perturbpf, opfdata, opflearn, df_pg, df_dk
):
    # Group data by bus
    bus_groups_df_dk = df_dk.groupby("bus")

    bus_groups_df_pg = df_pg.groupby("bus")

    bus_data_pfdelta = []  # pfdelta
    bus_data_opfdata = []  # opfdata
    bus_data_opflearn = []  # opflearn

    for i in sample_buses:  # Iterate over each bus
        bus_data_pfdelta.append(perturbpf[feature][:, i])
        bus_data_opfdata.append(opfdata[feature][:, i])
        bus_data_opflearn.append(opflearn[feature][:, i])

    bus_data_dk = [
        bus_groups_df_dk.get_group(bus)[feature].values for bus in sample_buses
    ]
    bus_data_pg = [
        bus_groups_df_pg.get_group(bus)[feature].values for bus in sample_buses
    ]

    datasets = [
        bus_data_dk,
        bus_data_opfdata,
        bus_data_pfdelta,
        bus_data_opflearn,
        bus_data_pg,
    ]

    # Create the plot
    fig, ax = plt.subplots(figsize=(15, 6))

    x_pos = np.array(
        [np.arange(len(sample_buses)) + i * 0.2 for i in range(5)]
    ).T.flatten()

    # Plot
    colors = ["lightgreen", "pink", "thistle", "lightblue", "lightyellow"]

    for i, data in enumerate(datasets):
        bp = ax.boxplot(
            data,
            sym="",
            whis=[0, 100],
            widths=0.15,
            showfliers=False,
            showcaps=True,
            patch_artist=True,
            medianprops=dict(color="black", linewidth=1.5),
            tick_labels=list(datasets[0]),
            positions=[x_pos[i] + j * 1 for j in range(len(data))],
        )

        for pc in bp["boxes"]:
            pc.set_facecolor(colors[i % 5])  # Cycles through colors
            pc.set_alpha(0.6)  # Transparency

    # Titles
    ax.set(
        title=f"{feature} distribution across buses", ylabel=feature, xlabel="Bus Index"
    )
    # Remove the major x-axis tickmarks
    ax.tick_params(axis="x", bottom=False)
    xtick_positions = np.arange(len(sample_buses)) + 0.2
    ax.set_xticks(xtick_positions)

    # Positions of the x-axis labels
    ax.set_xlabel("Bus Index")
    ax.set_ylabel(feature)

    # Positions of the minor x-axis tickmarks
    ax.set_xticklabels(
        [f"Bus {bus}" for bus in sample_buses],
        # rotation=45,
        ha="center",
    )
    ax.grid(True, alpha=0.3)
    # Add legend
    group_labels = ["GFM-Datakit", "OPF: n-1", "Pfdelta", "OPF Learn", "PgLearn"]
    legend_patches = [
        mpatches.Patch(color=colors[i], label=group_labels[i]) for i in range(5)
    ]
    ax.legend(handles=legend_patches, loc="best")
    fig.canvas.header_visible = False
    plt.tight_layout()
    plt.show()

In [13]:
# Add widgets
toggle_button = widgets.ToggleButtons(
    options=["vm", "pd", "qd", "pg", "qg"],
    description="Feature",
    disabled=False,
    button_style="",  # 'success', 'info', 'warning', 'danger' or ''
    tooltips=["vm", "pd", "qd", "pg", "qg"],
    # layout=Layout(width="50%", height='30px', padding='10px')
)
# Number of buses
text_input = widgets.IntSlider(
    value=10,
    min=0,
    max=54,
    step=1,
    description="# buses to display:",
    orientation="horizontal",
    disabled=False,
)
# display(text_input)

# Create a Dropdown widget
dropdown = widgets.Dropdown(
    options=[
        "pf_no_gc",
        "pf_gc_no_topology",
        "pf_gc_random_topology",
        "contingency_gc_random_topology",
    ],
    value="pf_no_gc",  # Initial selected value
    description="Select data:",
)

In [14]:
def f(dropdown_value, input_buses, toggle_val):
    plt.clf()
    if toggle_val == "pd" or toggle_val == "qd":
        sample_buses = np.sort(random.sample(range(99), input_buses))
    elif toggle_val == "vm":
        sample_buses = np.sort(random.sample(range(118), input_buses))
    elif toggle_val == "pg" or toggle_val == "qg":
        sample_buses = np.sort(random.sample(range(54), input_buses))

    if dropdown_value == "pf_no_gc":
        plotting_function(
            toggle_val, sample_buses, perturbpf, opfdata, opflearn, df_pg, df_dk_no_gc
        )
    elif dropdown_value == "pf_gc_no_topology":
        plotting_function(
            toggle_val,
            sample_buses,
            perturbpf,
            opfdata,
            opflearn,
            df_pg,
            datakit_gc_no_topology,
        )
    elif dropdown_value == "pf_gc_random_topology":
        plotting_function(
            toggle_val,
            sample_buses,
            perturbpf,
            opfdata,
            opflearn,
            df_pg,
            datakit_gc_random_topology,
        )
    elif dropdown_value == "contingency_gc_random_topology":
        plotting_function(
            toggle_val,
            sample_buses,
            perturbpf,
            opfdata,
            opflearn,
            df_pg,
            datakit_gc_random_topology_contingency,
        )

In [15]:
interactive_plot = widgets.interactive(
    f, dropdown_value=dropdown, input_buses=text_input, toggle_val=toggle_button
)
dropdown
text_input
toggle_button
nwidget_vbox = widgets.VBox([toggle_button])

display(interactive_plot)

interactive(children=(Dropdown(description='Select data:', options=('pf_no_gc', 'pf_gc_no_topology', 'pf_gc_ra…